In [4]:
from models import load_all_models
model_lst = load_all_models()

Loaded detection model vikp/surya_det3 on device mps with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device mps with dtype torch.float16
Loaded reading order model vikp/surya_order on device mps with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device mps with dtype torch.float16
Loaded texify model to mps with torch.float16 dtype
Loaded recognition model vikp/surya_tablerec on device mps with dtype torch.float16


## Single file

In [2]:
from convert import convert_single_pdf
fpath="/Users/jennyxu/Downloads/Archive 2/scm_metals/NCS000050_Permit (Issuance)_20230524.pdf"
full_text, doc_images, out_meta, pages, text_blocks = convert_single_pdf(fpath, model_lst)

Recognizing Text:   0%|          | 0/1 [00:01<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.51 GiB. GPU 0 has a total capacity of 23.55 GiB of which 1.46 GiB is free. Process 10966 has 1.04 GiB memory in use. Process 52004 has 5.96 GiB memory in use. Including non-PyTorch memory, this process has 15.02 GiB memory in use. Of the allocated memory 12.53 GiB is allocated by PyTorch, and 2.03 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [19]:
[[block.id, block.bbox, block.text, block.block_type, block.page_end] for i, block in enumerate(text_blocks)]

[['1|-7',
  [184.0, 94.04461319411486, 728.0, 679.3222591362127],
  'Division of Waste Management December 23, 1997 Mr. Phil Vorsatz\n\n![0_image_0.png](0_image_0.png)\n\n\n\n![0_image_1.png](0_image_1.png)\n\n\n\n![0_image_2.png](0_image_2.png)\n\n\nNC CERCLA Project Officer EPA Region IV Waste Division 345 Courtland Street, NE\nAtlanta, Georgia 30365 RE:\nBrownfield Site Assessment Report Eakes Dry Cleaners.\n\nDurham, Durham County, North Carolina Dear Mr. Vorsatz:\nThis letter is being sent to you to convey the information that the North Carolina Superfund Section has obtained on a site named Eakes Dry Cleaners.\n\nEakes Cleaners is located at 827 West Morgan Street in Durham, North Carolina.  It consists of a 6,600 square foot brick and block building situated on approximately 0.3 acres. The site was a Dry Cleaning and Fur Storage business for many years until its closure in 1995, after the death of Mr. Eakes and the bankruptcy of his estate. The site is located in a mixed commerc

In [9]:
import os
allfn = os.listdir("/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447")
pdfs = set([fn for fn in allfn if fn.endswith(".pdf")])
mds = set([fn for fn in allfn if not fn.endswith(".pdf")])

## Pipeline

In [7]:
import os
from convert import convert_single_pdf
from marker.schema.page import Page
from output import save_markdown
import pandas as pd

def page_to_dict(page: Page) -> dict:
    # Manually convert Page to dictionary if necessary
    return {
        'bbox':page.bbox,
        'pnum': page.pnum,
        'rotation': page.rotation,
        'text_lines': page.text_lines,  # Assuming text_lines can be converted
        'layout': page.layout,          # Assuming layout can be converted
        'order': page.order,
        'ocr_method': page.ocr_method,
        'char_blocks': page.char_blocks,  # Assuming lists of dicts can be converted
        'images': page.images,           # Handle images if needed
    }
def get_dir_pdfs(parent_dir):
    if not os.path.exists(parent_dir):
        raise Exception("Starting directory '%s' does not exist" % parent_dir)
    return [fn for fn in os.listdir(parent_dir) if ".pdf" in fn]
def to_refdocid(s):
    return (
        s.replace(".pdf","").replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "_")
        .replace(".", "-")
        .replace("#", "")
        .replace("'", "")
    )
def rename_pdf(parent_dir, filename, new_filename):
    old_file = os.path.join(parent_dir, filename)
    new_file = os.path.join(parent_dir, new_filename)
    os.rename(old_file, new_file)
    
def run(parent_dir, output_fn, override_existing=False):
    pdf_fns = get_dir_pdfs(parent_dir)
    all_tblocks = []
    all_pages = []
    error_files = []
    for fn in pdf_fns:
        docid = to_refdocid(fn)
        rename_pdf(parent_dir, fn, docid + ".pdf")
        input_path = os.path.join(parent_dir, docid + ".pdf")
        if override_existing is False and os.path.exists(input_path.replace(".pdf", "")):
            continue
        try:
            full_text, images, out_meta, pages, tblocks = run_single(input_path, docid)
            subfolder_path = save_markdown(parent_dir, fn, full_text, images, out_meta)
            pd.DataFrame(tblocks, columns=['id', 'bbox', 'text', 'type', 'page_end']).to_csv(f"{subfolder_path}/{docid}_blocks.csv")
            page_data = [page_to_dict(page) for page in pages]
            pd.DataFrame(page_data).to_csv(f"{subfolder_path}/{docid}_pages.csv")
            all_pages.append([docid, page_data[0]['bbox']])
            pd.DataFrame(all_pages).to_csv(f"{output_fn}_pages.csv")
            print(f"Processed {len(tblocks)} blocks for {docid}")
            all_tblocks.extend(tblocks)
            pd.DataFrame(all_tblocks, columns=['id', 'bbox', 'text', 'type', 'page_end']).to_csv(f"{output_fn}_blocks.csv")
        except Exception as e:
            print(f"Error processing {input_path}: {e}")
            error_files.append(docid)
    return all_tblocks, all_pages, error_files

def run_single(input_path, docid):
    full_text, images, out_meta, pages, text_blocks = convert_single_pdf(input_path, model_lst)
    tblocks = [[f"{docid}-|{block.id}", block.bbox, block.text, block.block_type, block.page_end] for i, block in enumerate(text_blocks)]
    return full_text, images, out_meta, pages, tblocks

In [8]:
all_blocks, all_pages, errors = run("/Users/jennyxu/Downloads/Archive 2/scm_metals", "NCS000050")

Recognizing Text:   0%|          | 0/3 [00:07<?, ?it/s]


Error processing /Users/jennyxu/Downloads/Archive 2/scm_metals/NCS000050_Permit_Issuance_19940831.pdf: MPS backend out of memory (MPS allocated: 8.66 GB, other allocations: 9.45 GB, max allowed: 18.13 GB). Tried to allocate 26.25 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).


Detecting bboxes:   0%|          | 0/4 [00:00<?, ?it/s]


Error processing /Users/jennyxu/Downloads/Archive 2/scm_metals/NCS000050_Renewal_Application_20160729.pdf: MPS backend out of memory (MPS allocated: 8.66 GB, other allocations: 9.45 GB, max allowed: 18.13 GB). Tried to allocate 32.96 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).


Detecting bboxes:   0%|          | 0/1 [00:00<?, ?it/s]


...

Processed 4 blocks for NCS000050_Email_RE_Renewal_Application_Amendments_20190314


Recognizing tables: 100%|██████████| 1/1 [00:03<00:00,  3.24s/it]


Processed 7 blocks for NCS000050_Email_RE_Cooling_Tower_Release_20211108


Recognizing Text:   0%|          | 0/1 [00:34<?, ?it/s]


KeyboardInterrupt: 